<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/%EC%8B%9D%EC%83%9D%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import json
import time

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [2]:
with open('SEORAKSAN.geojson') as f:
    geojson = json.load(f)
seoraksan = ee.FeatureCollection(geojson)

In [4]:
from ipyleaflet import TileLayer

# Vworld 배경지도 객체
vworld_base = TileLayer(
    url='https://xdworld.vworld.kr/2d/Base/service/{z}/{x}/{y}.png',
    name='Vworld Base',
    attribution='Vworld',
)

# 설악산국립공원 경계 가시화
m = geemap.Map(width="800px", height="500px")
m.add_layer(vworld_base)
m.addLayer(seoraksan, {'color': 'green'}, 'Seoraksan') # 레이어 추가
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
def mask_s2_clouds(image):
    # QA(Quality Assurance) 밴드 사용, S2에서 구름 마스킹
    qa = image.select('QA60')

    # 비트 10은 구름(clouds), 11은 성층운(cirrus)
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # 구름과 성층운이 0이면 맑은 상태로 간주함.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask).divide(10000) # 스케일링

In [6]:
# Sentinel-2 이미지 선택 및 필터링
s2_images = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2024-01-01", "2024-01-31")
    .filterBounds(seoraksan)
    # 구름이 5% 미만인 이미지 필터링
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
    .map(mask_s2_clouds)
)

# 이미지 컬렉션의 이미지 개수 확인
image_count = s2_images.size()

# 이미지 개수 출력
print("Image count:", image_count.getInfo())

Image count: 4


In [7]:
# 중간값 이미지 계산
s2_image = s2_images.median()

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map(width="800px", height="500px")
m.add_layer(s2_image, visualization, 'RGB')
m.centerObject(seoraksan, 11) # 지도의 중심 설정
m # 지도 객체 출력

Map(center=[38.136218061277916, 128.41248414920196], controls=(WidgetControl(options=['position', 'transparent…

In [19]:
# 밴드 이름 목록
band_names = s2_image.bandNames()
print(band_names.getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


In [20]:
# B2, B3, B4, B8 밴드 선택
rgb_image = s2_image.select(['B2', 'B3', 'B4']).clip(seoraksan.geometry())

In [21]:
# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=rgb_image,
    description='S2_RGB_202401',
    folder='export',
    scale=10,  # 이미지의 해상도
    region=seoraksan.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: READY
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: RUNNING
Polling for task (id: 5Y4TQT37GUBWG7Z77BTGL6SB). Status: COMPLETED
Task completed. Final status:
{'state': 'COMPLETED', 'description': 'S2_RGB_202401', 'priority': 100, 'creation_timestamp_ms': 1719820089773, 'update_timestamp_ms': 1719820405253, 

In [22]:
# NDVI 계산 (NIR = B8, Red = B4)
ndvi_image = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI').clip(seoraksan.geometry())

In [23]:
# 이미지를 Google Drive에 내보내기
task = ee.batch.Export.image.toDrive(
    image=ndvi_image,
    description='S2_NDVI_202401',
    folder='export',
    scale=10,  # 이미지의 해상도
    region=seoraksan.geometry(),  # 내보낼 영역
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

# 내보내기 작업 시작
task.start()

# 내보내기 작업 상태 확인
print('Export task started. Checking status...')

while True:
    status = task.status()
    state = status['state']
    print('Polling for task (id: {}). Status: {}'.format(task.id, state))
    if state in ['COMPLETED', 'FAILED']:
        break
    time.sleep(30)  # 30초 간격으로 상태 확인

# 완료 후 상태 출력
print('Task completed. Final status:')
print(status)

Export task started. Checking status...
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: READY
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: RUNNING
Polling for task (id: P7CUOAYPVU4TBG4U25BS7I7T). Status: COMPLETED
Task completed. Final status:
{'state': 'COMPLETED

In [25]:
# NDVI 값을 기준으로 마스크 적용
# ndvi_mask = ndvi_image.gt(-0.1).And(ndvi_image.lt(0.1))
ndvi_mask = ndvi_image.lt(0.1)
masked_ndvi = ndvi_image.updateMask(ndvi_mask)

# seoraksan 영역 내에서 마스크된 이미지의 픽셀 수 계산
area_image = masked_ndvi.multiply(ee.Image.pixelArea())
area = area_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=seoraksan.geometry(),
    scale=10,
    maxPixels=1e13
)

# 결과 면적 출력
area_value_sqm = area.getInfo()['NDVI']  # NDVI 밴드의 면적 값 가져오기
area_value_sqkm = area_value_sqm / 1e6  # 제곱킬로미터로 변환
print('NDVI 값이 -0.1보다 크고 0.1보다 작은 면적:', area_value_sqkm, 'square kilometers')

NDVI 값이 -0.1보다 크고 0.1보다 작은 면적: 4.905465367498134 square kilometers
